# Topological K-means

#### Importing libraries

In [34]:
import numpy as np # Linear algebra
import pandas as pd # Data processing
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # Data visualization
import networkx as nx # Graph analysis and manipulation

from numpy import dot
from numpy import trace
from numpy.linalg import inv
import scipy.sparse._csr
%matplotlib inline

import sklearn.datasets as skdata
import sklearn.neighbors as sknn
import sklearn.utils.graph as sksp
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import silhouette_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import calinski_harabasz_score
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.metrics.cluster import davies_bouldin_score

import sys
import time
import warnings
warnings.simplefilter(action='ignore')

#### Importing dataset

In [2]:
### Scikit-learn datasets
# (>) Melhor (=) Próximos (<) Pior

#X = skdata.fetch_openml(name='penguins', version=1)                     # (344, 6)      - NaNs
#X = skdata.load_iris()                                                  # (150, 4)      - (<)
#X = skdata.fetch_openml(name='segment', version=2)                      # (2310, 19)    - (<)
#X = skdata.fetch_openml(name='amazon-commerce-reviews', version=1)      # (1500, 10000) - (<)
#X = skdata.fetch_openml(name='semeion', version=1)                      # (1593, 256)   - (<)
#X = skdata.fetch_openml(name='mfeat-pixel', version=3)                  # (2000, 240)   - (<)
#X = skdata.fetch_openml(name='micro-mass', version=1)                   # (360, 1300)   - (<)
#X = skdata.fetch_openml(name='monks-problems-1', version=1)             # (556, 6)      - (<)
#X = skdata.fetch_openml(name='breast-tissue', version=1)                # (106, 9)      - (<)
#X = skdata.fetch_openml(name='fri_c2_100_10', version=2)                # (100, 10)     - (<)
#X = skdata.fetch_openml(name='datatrieve', version=1)                   # (130, 8)      - (<)
#X = skdata.fetch_openml(name='fri_c3_250_25', version=2)                # (250, 25)     - (<)
#X = skdata.fetch_openml(name='cnae-9', version=2)                       # (240, 856)    - Bem pior (<)
#X = skdata.fetch_openml(name='oh5.wc', version=1)                       # (918, 3012)   - Bem pior (<)
#X = skdata.fetch_openml(name='GCM', version=1)                          # (190, 16063)  - (=)
#X = skdata.fetch_openml(name='collins', version=2)                      # (500, 22)     - (=)
#X = skdata.fetch_openml(name='pyrim', version=2)                        # (74, 27)      - (=)
#X = skdata.fetch_openml(name='balance-scale', version=1)                # (625, 4)      - (=)
#X = skdata.fetch_openml(name='tr45.wc', version=1)                      # (690, 8261)   - (>)
#X = skdata.fetch_openml(name='cloud', version=2)                        # (108, 7)      - (>)
#X = skdata.fetch_openml(name='servo', version=1)                        # (167, 4)      - (>)
#X = skdata.fetch_openml(name='AP_Breast_Lung', version=1)               # (470, 10935)  - Muito bom! (>)
#X = skdata.fetch_openml(name='leukemia', version=1)                     # (72, 7129)    - Excelente! (>)
#X = skdata.fetch_openml(name='AP_Colon_Prostate', version=1)            # (355, 10935)  - Excelente! (>)
#X = skdata.fetch_openml(name='AP_Colon_Kidney', version=1)              # (546, 10935)  - Excelente! (>)      
X = skdata.fetch_openml(name='AP_Breast_Kidney', version=1)
# X = skdata.fetch_openml(name='AP_Breast_Ovary', version=1)
# X = skdata.fetch_openml(name='AP_Breast_Colon', version=1)   
# X = skdata.fetch_openml(name='AP_Breast_Lung', version=1)        
# X = skdata.fetch_openml(name='AP_Endometrium_Breast', version=1)        
# X = skdata.fetch_openml(name='AP_Colon_Omentum', version=1)        
# X = skdata.fetch_openml(name='AP_Breast_Omentum', version=1)        
# X = skdata.fetch_openml(name='AP_Prostate_Kidney', version=1)        
# X = skdata.fetch_openml(name='AP_Omentum_Kidney', version=1)        
# X = skdata.fetch_openml(name='AP_Breast_Prostate', version=1)
# X = skdata.fetch_openml(name='AP_Uterus_Kidney', version=1)
# X = skdata.fetch_openml(name='AP_Prostate_Uterus', version=1)
# X = skdata.fetch_openml(name='AP_Omentum_Lung', version=1)
# X = skdata.fetch_openml(name='AP_Lung_Uterus', version=1)
# X = skdata.fetch_openml(name='AP_Ovary_Kidney', version=1)
# X = skdata.fetch_openml(name='AP_Lung_Kidney', version=1)
# X = skdata.fetch_openml(name='AP_Colon_Lung', version=1)
# X = skdata.fetch_openml(name='AP_Endometrium_Colon', version=1)
# X = skdata.fetch_openml(name='AP_Ovary_Lung', version=1)
# X = skdata.fetch_openml(name='AP_Endometrium_Kidney', version=1)
# X = skdata.fetch_openml(name='AP_Colon_Uterus', version=1)
# X = skdata.fetch_openml(name='leukemia', version=1)                
# X = skdata.fetch_openml(name='tr11.wc', version=1)
# X = skdata.fetch_openml(name='tr12.wc', version=1)
# X = skdata.fetch_openml(name='tr45.wc', version=1)
# X = skdata.fetch_openml(name='tr23.wc', version=1)
# X = skdata.fetch_openml(name='tr21.wc', version=1)
# X = skdata.fetch_openml(name='tr31.wc', version=1)
# X = skdata.fetch_openml(name='hepatitisC', version=1)
# X = skdata.fetch_openml(name='Ovarian', version=1)
# X = skdata.fetch_openml(name='SRBCT', version=1)
# X = skdata.fetch_openml(name='Colon', version=1)
# X = skdata.fetch_openml(name='cloud', version=2)              
# X = skdata.fetch_openml(name='servo', version=1)        
# X = skdata.fetch_openml(name='climate-model-simulation-crashes', version=1)
# X = skdata.fetch_openml(name='anneal', version=1)
# X = skdata.fetch_openml(name='pasture', version=2)
# X = skdata.fetch_openml(name='analcatdata_authorship', version=2)
# X = skdata.fetch_openml(name='stock', version=2)
# X = skdata.fetch_openml(name='glass', version=2)
# X = skdata.fetch_openml(name='lowbwt', version=2)
# X = skdata.fetch_openml(name='kc1-binary', version=1)
# X = skdata.fetch_openml(name='analcatdata_marketing', version=2)
# X = skdata.fetch_openml(name='boston', version=2)
# X = skdata.fetch_openml(name='dermatology', version=2)
# X = skdata.fetch_openml(name='backache', version=1)
# X = skdata.fetch_openml(name='lsvt', version=1)
# X = skdata.fetch_openml(name='thoracic-surgery', version=1)
# X = skdata.fetch_openml(name='planning-relax', version=1)

### Lentos
#X = skdata.load_digits()                                                # (1797, 64)    - (<)
#X = skdata.fetch_openml(name='phoneme', version=1)                      # (5404, 5)     - (<)
#X = skdata.fetch_openml(name='Bioresponse', version=1)                  # (3751, 1776)  -
#X = skdata.fetch_openml(name='optdigits', version=1)                    # (5620, 64)    -
#X = skdata.fetch_openml(name='satimage', version=1)                     # (6430, 36)    -

y = X['target']
X = X['data']
k = len(np.unique(y)) # Number of unique labels
n, m = X.shape # Number of observations and variables

if type(X) == scipy.sparse._csr.csr_matrix:
    X = X.todense()
    X = np.asarray(X)
else:
    if not isinstance(X, np.ndarray):
        cat_cols = X.select_dtypes(['category']).columns
        X[cat_cols] = X[cat_cols].apply(lambda x: x.cat.codes)
        X = X.to_numpy()
        y = y.to_numpy()

#### Exploratory Data Analysis

In [3]:
df = pd.DataFrame(data=X)
df.shape # (observations, variables)

(604, 10935)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,10925,10926,10927,10928,10929,10930,10931,10932,10933,10934
0,3930.3,519.8,1787.3,896.9,157.8,928.5,2670.0,489.6,193.9,491.1,...,3384.2,8452.7,7491.5,38134.3,34646.4,114629.7,83935.7,1739.8,907.2,1026.0
1,4497.3,564.4,1492.9,1056.7,139.8,1740.7,1874.4,262.7,127.7,221.0,...,1361.0,1376.7,1300.5,20786.1,18601.8,63899.7,47226.0,1583.5,594.9,912.9
2,2877.2,449.4,1788.0,694.7,149.1,1356.4,1255.4,335.4,121.9,115.2,...,1452.3,4646.1,3416.6,17594.9,16745.1,50491.2,38088.0,2414.1,710.1,1338.9
3,2503.7,669.0,509.8,664.5,277.2,1309.8,1104.0,397.4,266.3,280.6,...,4151.1,10018.5,10653.2,54989.5,46099.9,125395.9,99534.9,3588.1,1404.8,2013.6
4,2240.8,508.0,2121.5,418.9,76.0,677.1,966.5,512.3,233.8,210.2,...,1084.5,3022.9,2530.0,14794.1,13729.8,40181.3,29450.5,751.3,189.2,392.5


In [5]:
print(np.unique(y))
print('Number of unique labels: {}'.format(k))

['Breast' 'Kidney']
Number of unique labels: 2


#### Converting labels into integers

In [6]:
labels = list(np.unique(y))
numbers = np.zeros(n)
for i in range(n):
    numbers[i] = labels.index(y[i])

#### Topological K-means implementation

In [7]:
def centroid_distances(G, coord_centroids):
    distances = []
    for i in coord_centroids:
        D = nx.single_source_dijkstra(G, i)[0]
        temp = np.ones(n) * np.inf
        for j in D.keys():
            temp[j] = D[j]
        distances.append(temp)

    return np.array(distances)

def TopologicalKMeans(X):
    n_sqrt = round(np.sqrt(n))
    
    coord_centroids = np.random.choice(n, size=k, replace=False)
    centroids = X[coord_centroids] # Randomly chosen observations as centroids

    iteration = 0
    while iteration < 10:
        iteration += 1
        
        knnGraph = sknn.kneighbors_graph(X, n_neighbors=n_sqrt, mode='distance') # KNN graph
        W = knnGraph.toarray() # Adjacency matrix
        
        G = nx.from_numpy_array(W) # NetworkX graph
        distances = centroid_distances(G,coord_centroids) # Geodesic distances from each centroid to all observations

        geo_labels = np.zeros(n) # Array for labels
        for j in range(n):
            geo_labels[j] = distances[:, j].argmin() # Updating labels
        
        new_centroids = np.zeros((k, m))
        for r in range(k):
            indices = np.where(geo_labels==r)[0]
            
            if len(indices) > 0:
                sample = X[indices]
                new_centroids[r, :] = sample.mean(axis=0)
            else:
                new_centroids[r, :] = centroids[r, :]
        
        if (np.linalg.norm(centroids - new_centroids) < 0.05):
            break

        centroids = new_centroids.copy() # Updated centroids
    
    return geo_labels

#### Comparação dos algoritmos

In [45]:
def kmeans_exec(X, iterations:int, mode='classical'):
    if mode not in ['classical','topological']:
        raise ValueError("Mode argument must be 'classical' or 'topological'.")

    rand_list, sc_list, ami_list, ch_list, fm_list, db_list = [], [], [], [], [], []
    start_time = time.time()
    
    if mode == 'classical':
        for i in range(1, iterations+1):
            if (i == 1) or (i % 5 == 0):
                print('Iteration: {}/{}'.format(i, iterations))
            kmeans = KMeans(n_clusters=k, init='random', n_init=1).fit(X)
            rand_list.append(rand_score(numbers, kmeans.labels_))
            ami_list.append(adjusted_mutual_info_score(numbers, kmeans.labels_))
            fm_list.append(fowlkes_mallows_score(numbers, kmeans.labels_))
            sc_list.append(silhouette_score(X, kmeans.labels_))
            ch_list.append(calinski_harabasz_score(X, kmeans.labels_))
            db_list.append(davies_bouldin_score(X, kmeans.labels_))
        
        print()
        print('Classical K-means')
    
    else:
        for i in range(1, iterations+1):
            if (i == 1) or (i % 5 == 0):
                print('Iteration: {}/{}'.format(i, iterations))
            labels = TopologicalKMeans(X)
            rand_list.append(rand_score(numbers, labels))
            ami_list.append(adjusted_mutual_info_score(numbers, labels))
            fm_list.append(fowlkes_mallows_score(numbers, labels))
            sc_list.append(silhouette_score(X, labels))
            ch_list.append(calinski_harabasz_score(X, labels))
            db_list.append(davies_bouldin_score(X, labels))

        print()
        print('Topological K-means')
    
    end_time = time.time()
    print('Elapsed time: %.4f' %(end_time - start_time))
    print()
    print('Average Rand score: %.4f' %(np.mean(rand_list)))             # Upper bound of 1
    print('Average AMI score: %.4f' %(np.mean(ami_list)))               # Upper bound of 1
    print('Average Fowlkes-Mallows score: %.4f' %(np.mean(fm_list)))    # Upper bound of 1
    print('Average Silhouette coefficient: %.4f' %(np.mean(sc_list)))   # Bounded between -1 and +1; scores around 0 indicate overlapping clusters
    print('Average Calinski-Harabasz score: %.4f' %(np.mean(ch_list)))
    print('Average Davies-Bouldin score: %.4f' %(np.mean(db_list)))     # Values closer to 0 indicate a better partition

##### Classical K-means

In [46]:
kmeans_exec(X, 30, mode='classical')

Iteration: 1/30
Iteration: 5/30
Iteration: 10/30
Iteration: 15/30
Iteration: 20/30
Iteration: 25/30
Iteration: 30/30

Classical K-means
Elapsed time: 10.6276

Average Rand score: 0.5132
Average AMI score: 0.0156
Average Fowlkes-Mallows score: 0.5303
Average Silhouette coefficient: 0.1685
Average Calinski-Harabasz score: 104.9210
Average Davies-Bouldin score: 2.1920


##### Topological K-means

In [47]:
kmeans_exec(X, 30, mode='topological')

Iteration: 1/30
Iteration: 5/30
Iteration: 10/30
Iteration: 15/30
Iteration: 20/30
Iteration: 25/30
Iteration: 30/30

Topological K-means
Elapsed time: 20.0121

Average Rand score: 0.6617
Average AMI score: 0.2901
Average Fowlkes-Mallows score: 0.6955
Average Silhouette coefficient: 0.1027
Average Calinski-Harabasz score: 58.9166
Average Davies-Bouldin score: 2.8931
